In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

## Chargement des données

In [3]:
print("loading data...")

train_transaction = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/train_transaction.csv', index_col='TransactionID')
test_transaction = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/train_identity.csv', index_col='TransactionID')
test_identity = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/test_identity.csv', index_col='TransactionID')

print("loading successful!")

loading data...
loading successful!


## Isolation de Y

In [4]:
print("Number of missing values in target : ", train_transaction.isFraud.isnull().sum(), "\n")  # 0 missing values in target
print("percent of fraudulent train-transactions: ", len(train_transaction.loc[train_transaction.isFraud == 1])*100/len(train_transaction))

Number of missing values in target :  0 

percent of fraudulent train-transactions:  3.499000914417313


In [5]:
print("Shape train_transaction avant drop : ", train_transaction.shape, "\n")
y_train = train_transaction["isFraud"]
train_transaction = train_transaction.drop(columns = ['isFraud'])

print("Shape y_train : ", y_train.shape, "\n")
print("Shape train_transaction avant drop : ", train_transaction.shape, "\n")

Shape train_transaction avant drop :  (590540, 393) 

Shape y_train :  (590540,) 

Shape train_transaction avant drop :  (590540, 392) 



## On renomme les données car les colonnes de train et test ne sont pas les mêmes, les colonnes de test ont des tirets au lieu d'underscore.



In [6]:
# On rename les colonnes de test_identity selon train_identity
test_identity.columns = ['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08',
       'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16',
       'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24',
       'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo']


print("before concatting: \n\n")
print("train_transaction.shape: ", train_transaction.shape, "\n")
print("test_transaction.shape: ", test_transaction.shape, "\n")
print("train_transaction.index: ", train_transaction.index, "\n")
print("test_transaction.index: ", test_transaction.index, "\n")

print("train_identity.shape: ", train_identity.shape, "\n")
print("test_identity.shape: ", test_identity.shape, "\n")
print("train_identity.index: ", train_identity.index, "\n")
print("test_identity.index: ", test_identity.index, "\n")


transaction_data = pd.concat([train_transaction, test_transaction])
identity_data = pd.concat([train_identity, test_identity])


print("after concatting: \n\n")
print("transaction_data.shape: ", transaction_data.shape, "\n")
print("transaction_data.index: ", transaction_data.index, "\n")
print("identity_data.shape: ", identity_data.shape, "\n")
print("identity_data.index: ", identity_data.index, "\n")

before concatting: 


train_transaction.shape:  (590540, 392) 

test_transaction.shape:  (506691, 392) 

train_transaction.index:  Int64Index([2987000, 2987001, 2987002, 2987003, 2987004, 2987005, 2987006,
            2987007, 2987008, 2987009,
            ...
            3577530, 3577531, 3577532, 3577533, 3577534, 3577535, 3577536,
            3577537, 3577538, 3577539],
           dtype='int64', name='TransactionID', length=590540) 

test_transaction.index:  Int64Index([3663549, 3663550, 3663551, 3663552, 3663553, 3663554, 3663555,
            3663556, 3663557, 3663558,
            ...
            4170230, 4170231, 4170232, 4170233, 4170234, 4170235, 4170236,
            4170237, 4170238, 4170239],
           dtype='int64', name='TransactionID', length=506691) 

train_identity.shape:  (144233, 40) 

test_identity.shape:  (141907, 40) 

train_identity.index:  Int64Index([2987004, 2987008, 2987010, 2987011, 2987016, 2987017, 2987022,
            2987038, 2987040, 2987048,
            

## CLASSEMENT DES NAN

on classe les colonnes en fonction de leur quantité de NAN et de leur type numérique ou categorique

### *Identity*

In [7]:
c = (identity_data.dtypes == 'object') #categorique
n = (identity_data.dtypes != 'object') #numerique

#on récupère le nom des colonnes spécifique à chaque type
cat_id_cols = list(c[c].index)
num_id_cols = list(n[n].index) 

print(cat_id_cols, "\n")
print("number categorical identity features: ", len(cat_id_cols), "\n\n")
print(num_id_cols, "\n")
print("number numerical identity features: ", len(num_id_cols))

['id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo'] 

number categorical identity features:  17 


['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_13', 'id_14', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_24', 'id_25', 'id_26', 'id_32'] 

number numerical identity features:  23


### *Transaction*

In [8]:
c = (transaction_data.dtypes == 'object') #categorique
n = (transaction_data.dtypes != 'object') #numerique

#on récupère le nom des colonnes de chaque type
cat_trans_cols = list(c[c].index)
num_trans_cols = list(n[n].index) 

print(cat_trans_cols, "\n")
print("number categorical transaction features: ", len(cat_trans_cols), "\n\n")
print(num_trans_cols, "\n")
print("number numerical transaction features: ", len(num_trans_cols))

['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9'] 

number categorical transaction features:  14 


['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 

In [9]:
# on garde la forme en mémoire des tableaux pour pouvoir les reconstruire après
shape_of_train_trans = train_transaction.shape
shape_of_train_id    = train_identity.shape

shape_of_test_trans  = test_transaction.shape
shape_of_test_id     = test_identity.shape

#on supprime les tableaux car on en a plus besoin
del train_transaction
del train_identity
del test_transaction
del test_identity

print("deletion successful!")

deletion successful!


### *Identity categorielle*

In [10]:
#on affiche le nombre de données manquantes pour les variales categorielles
for i in cat_id_cols:
    print(identity_data[i].value_counts())
    print(i, "missing values: ", identity_data[i].isnull().sum())
    print(identity_data[i].isnull().sum()*100/ len(identity_data[i]), "\n")

NotFound    243920
Found        42220
Name: id_12, dtype: int64
id_12 missing values:  0
0.0 

Found      135690
New        119397
Unknown     22875
Name: id_15, dtype: int64
id_15 missing values:  8178
2.8580415181379744 

Found       132805
NotFound    122282
Name: id_16, dtype: int64
id_16 missing values:  31053
10.852379953868736 

IP_PROXY:TRANSPARENT    7203
IP_PROXY:ANONYMOUS      2010
IP_PROXY:HIDDEN         1018
Name: id_23, dtype: int64
id_23 missing values:  275909
96.42447752848256 

Found       10214
NotFound       17
Name: id_27, dtype: int64
id_27 missing values:  275909
96.42447752848256 

Found    151813
New      125943
Name: id_28, dtype: int64
id_28 missing values:  8384
2.930034248969036 

Found       149264
NotFound    128492
Name: id_29, dtype: int64
id_29 missing values:  8384
2.930034248969036 

Windows 10          42170
Windows 7           23478
iOS 12.1.0           6349
Mac OS X 10_12_6     3884
iOS 11.2.1           3824
                    ...  
Mac OS X 10_1

In [11]:
#on sépare les variables en 3 categories : celles ou il manque beaucoup de données, un peu et pas beaucoup
low_missing_cat_id_cols = []      # lower than 15% missing values
medium_missing_cat_id_cols = []   # between 15% and 60% missing
many_missing_cat_id_cols = []     # more than 60% missing

#on tri les données en fonction du nombres de données manquantes
for i in cat_id_cols:
    percentage = identity_data[i].isnull().sum() * 100 / len(identity_data[i])
    if percentage < 15:
        low_missing_cat_id_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_cat_id_cols.append(i)
    else:
        many_missing_cat_id_cols.append(i)
        
print("cat_id_cols: \n\n")      
print("number low missing: ", len(low_missing_cat_id_cols), "\n")
print("number medium missing: ", len(medium_missing_cat_id_cols), "\n")
print("number many missing: ", len(many_missing_cat_id_cols), "\n")

cat_id_cols: 


number low missing:  11 

number medium missing:  4 

number many missing:  2 



### Identity *numérique*

In [12]:
#on affiche le nombre de données manquantes pour les variales numériques
for i in num_id_cols:
    print(identity_data[i].value_counts())
    print(i, "missing values: ", identity_data[i].isnull().sum()) 
    print(identity_data[i].isnull().sum()*100/len(identity_data[i]), "\n") # missing percent

-5.0     164749
 0.0      31555
-10.0     23050
-20.0     22341
-15.0     12722
          ...  
-49.0         1
-89.0         1
-91.0         1
-24.0         1
-78.0         1
Name: id_01, Length: 89, dtype: int64
id_01 missing values:  0
0.0 

696.0       16
1083.0      15
552.0       15
1120.0      12
1141.0      12
            ..
204779.0     1
10957.0      1
204778.0     1
204777.0     1
38982.0      1
Name: id_02, Length: 198051, dtype: int64
id_02 missing values:  8292
2.8978821555881735 

 0.0     127811
 1.0       1645
 3.0       1442
 2.0        806
 5.0        264
 4.0        184
 6.0        134
-5.0         88
-6.0         84
-2.0         58
-4.0         56
-7.0         54
-8.0         41
-9.0         27
-10.0        26
-11.0        20
-12.0        19
-1.0         18
-3.0         12
 7.0          6
 9.0          4
-13.0         3
 10.0         1
 11.0         1
 8.0          1
Name: id_03, dtype: int64
id_03 missing values:  153335
53.58740476689732 

 0.0     131390
-5.0   

In [13]:
#on sépare les variables en 3 categories : celles ou il manque beaucoup de données, un peu et pas beaucoup
low_missing_num_id_cols = []      # lower than 15% missing values
medium_missing_num_id_cols = []   # between 15% and 60% missing
many_missing_num_id_cols = []     # more than 60% missing

#on tri les données en fonction du nombres de données manquantes
for i in num_id_cols:
    percentage = identity_data[i].isnull().sum() * 100 / len(identity_data[i])
    if percentage < 15:
        low_missing_num_id_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_num_id_cols.append(i)
    else:
        many_missing_num_id_cols.append(i)
        
print("num_id_cols: \n\n")        
print("number low missing: ", len(low_missing_num_id_cols), "\n")
print("number medium missing: ", len(medium_missing_num_id_cols), "\n")
print("number many missing: ", len(many_missing_num_id_cols), "\n")

num_id_cols: 


number low missing:  9 

number medium missing:  6 

number many missing:  8 



### *Transactions categorielle*

In [14]:
for i in cat_trans_cols:
    print(transaction_data[i].value_counts())
    print(i, transaction_data[i].isnull().sum(), "missing values")
    print(i, transaction_data[i].isnull().sum()*100/len(transaction_data[i]), "\n")  # missing percent

W    800657
C    137785
R     73346
H     62397
S     23046
Name: ProductCD, dtype: int64
ProductCD 0 missing values
ProductCD 0.0 

visa                719649
mastercard          347386
american express     16009
discover              9524
Name: card4, dtype: int64
card4 4663 missing values
card4 0.42497887864998346 

debit              824959
credit             267648
debit or credit        30
charge card            16
Name: card6, dtype: int64
card6 4578 missing values
card6 0.41723210518113324 

gmail.com           435803
yahoo.com           182784
hotmail.com          85649
anonymous.com        71062
aol.com              52337
comcast.net          14474
icloud.com           12316
outlook.com           9934
att.net               7647
msn.com               7480
sbcglobal.net         5767
live.com              5720
verizon.net           5011
ymail.com             4075
bellsouth.net         3437
yahoo.com.mx          2827
me.com                2713
cox.net               2657
optonline

In [15]:
#on sépare les variables en 3 categories : celles ou il manque beaucoup de données, un peu et pas beaucoup
low_missing_cat_trans_cols = []      # lower than 15% missing values
medium_missing_cat_trans_cols = []   # between 15% and 60% missing
many_missing_cat_trans_cols = []     # more than 60% missing

#on tri les données en fonction du nombres de données manquantes
for i in cat_trans_cols:
    percentage = transaction_data[i].isnull().sum() * 100 / len(transaction_data[i])
    if percentage < 15:
        low_missing_cat_trans_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_cat_trans_cols.append(i)
    else:
        many_missing_cat_trans_cols.append(i)
        
print("cat_trans_cols: \n\n")    
print("number low missing: ", len(low_missing_cat_trans_cols), "\n")
print("number medium missing: ", len(medium_missing_cat_trans_cols), "\n")
print("number many missing: ", len(many_missing_cat_trans_cols), "\n")

cat_trans_cols: 


number low missing:  4 

number medium missing:  8 

number many missing:  2 



### *Transactions numérique*

In [16]:
for i in num_trans_cols:
    print(transaction_data[i].value_counts())
    print(i, transaction_data[i].isnull().sum(), "missing values")
    print(i, transaction_data[i].isnull().sum()*100/len(transaction_data[i]), "\n")  # missing percent

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
0.0    798701
1.0    188126
2.0      7275
3.0      1375
4.0       186
7.0       174
5.0       130
6.0        19
Name: V93, dtype: int64
V93 101245 missing values
V93 9.227318586514599 

0.0    851784
1.0    144184
2.0        18
Name: V94, dtype: int64
V94 101245 missing values
V94 9.227318586514599 

0.0      947053
1.0       98923
2.0       28080
3.0       10744
4.0        4976
          ...  
861.0         1
862.0         1
863.0         1
462.0         1
511.0         1
Name: V95, Length: 881, dtype: int64
V95 314 missing values
V95 0.028617492579046708 

0.0       675033
1.0       177098
2.0        87428
3.0        48513
4.0        30540
           ...  
1371.0         1
1372.0         1
1373.0         1
1374.0         1
1168.0         1
Name: V96, Length: 1410, dtype: int64
V96 314 missing values
V96 0.028617492579046708 

0.0      839537
1.0      146804
2.0       50473
3.0       22411
4.0       11832
  

In [17]:
#on sépare les variables en 3 categories : celles ou il manque beaucoup de données, un peu et pas beaucoup
low_missing_num_trans_cols = []      # lower than 15% missing values
medium_missing_num_trans_cols = []   # between 15% and 60% missing
many_missing_num_trans_cols = []     # more than 60% missing

#on tri les données en fonction du nombres de données manquantes
for i in num_trans_cols:
    percentage = transaction_data[i].isnull().sum() * 100 / len(transaction_data[i])
    if percentage < 15:
        low_missing_num_trans_cols.append(i)
    elif percentage >= 15 and percentage < 60:
        medium_missing_num_trans_cols.append(i)
    else:
        many_missing_num_trans_cols.append(i)
        
print("num_trans_cols: \n\n")        
print("number low missing: ", len(low_missing_num_trans_cols), "\n")
print("number medium missing: ", len(medium_missing_num_trans_cols), "\n")
print("number many missing: ", len(many_missing_num_trans_cols), "\n")

num_trans_cols: 


number low missing:  176 

number medium missing:  35 

number many missing:  167 



In [18]:
# Summary so far:

# we have 2 dataframes:   transaction_data and identity_data

####################################################################
# features:

# transaction_data:     14 categorical and 378 numerical features
# identity_data:        17 categorical and  23 numerical features
####################################################################
# missing values:

# cat_trans_cols:      4 low,    8 medium,    2 many 
# num_trans_cols:    176 low,   35 medium,  167 many

# cat_id_cols:        11 low,    4 medium,    2 many 
# num_id_cols:         9 low,    6 medium,    8 many
####################################################################

## On transforme les données en fonctions de leur categories : Numérique

### Valeurs numériques avec un nombres trop important de NAN : on drop les colonnes


In [19]:
# transactions
print("shape before dropping num_trans_cols: ", transaction_data.shape, "\n")        
transaction_data = transaction_data.drop(columns = many_missing_num_trans_cols)
print("shape after dropping num_trans_cols: ", transaction_data.shape, "\n\n")    

#identity
print("shape before dropping num_id_cols: ", identity_data.shape, "\n")        
identity_data = identity_data.drop(columns = many_missing_num_id_cols)
print("shape after dropping num_id_cols: ", identity_data.shape, "\n")

shape before dropping num_trans_cols:  (1097231, 392) 

shape after dropping num_trans_cols:  (1097231, 225) 


shape before dropping num_id_cols:  (286140, 40) 

shape after dropping num_id_cols:  (286140, 32) 



On met à jour la liste des variables numérique suite à la suppression de certaines

In [20]:
#maj de transactions
n = (transaction_data.dtypes != 'object')
num_trans_cols = list(n[n].index) 

#maj de identity
n = (identity_data.dtypes != 'object')
num_id_cols = list(n[n].index) 

### Peu de valeurs manquantes --> on remplace par la moyenne



In [21]:
from sklearn.impute import SimpleImputer

### *Transactions*

In [22]:
print("index before imputation: ", transaction_data.index, "\n")
print("columns before imputation: ", transaction_data.columns, "\n")

print("starting imputation..... \n\n")
my_imputer = SimpleImputer(strategy = 'mean') #on choisi d'appliquer la moyenne
my_imputer.fit(transaction_data[low_missing_num_trans_cols])

#transformation
transaction_data[low_missing_num_trans_cols] = my_imputer.transform(transaction_data[low_missing_num_trans_cols])

print("index after imputation: ", transaction_data.index, "\n")
print("columns after imputation: ", transaction_data.columns, "\n")
print("values after imputing: ", transaction_data[low_missing_num_trans_cols], "\n")

index before imputation:  Int64Index([2987000, 2987001, 2987002, 2987003, 2987004, 2987005, 2987006,
            2987007, 2987008, 2987009,
            ...
            4170230, 4170231, 4170232, 4170233, 4170234, 4170235, 4170236,
            4170237, 4170238, 4170239],
           dtype='int64', name='TransactionID', length=1097231) 

columns before imputation:  Index(['TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2',
       'card3', 'card4', 'card5', 'card6', 'addr1',
       ...
       'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320',
       'V321'],
      dtype='object', length=225) 

starting imputation..... 


index after imputation:  Int64Index([2987000, 2987001, 2987002, 2987003, 2987004, 2987005, 2987006,
            2987007, 2987008, 2987009,
            ...
            4170230, 4170231, 4170232, 4170233, 4170234, 4170235, 4170236,
            4170237, 4170238, 4170239],
           dtype='int64', name='TransactionID', length=1097231) 

colu

### *Identity*

In [23]:
print("index before imputation: ", identity_data.index, "\n")
print("columns before imputation: ", identity_data.columns, "\n")

print("starting imputation....\n")
my_imputer = SimpleImputer(strategy = 'mean') 
my_imputer.fit(identity_data[low_missing_num_id_cols])

#transformation
identity_data[low_missing_num_id_cols] = my_imputer.transform(identity_data[low_missing_num_id_cols])

print("index after imputation: ", identity_data.index, "\n")
print("columns after imputation: ", identity_data.columns, "\n")

index before imputation:  Int64Index([2987004, 2987008, 2987010, 2987011, 2987016, 2987017, 2987022,
            2987038, 2987040, 2987048,
            ...
            4170224, 4170225, 4170226, 4170228, 4170229, 4170230, 4170233,
            4170234, 4170236, 4170239],
           dtype='int64', name='TransactionID', length=286140) 

columns before imputation:  Index(['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_09', 'id_10',
       'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_19',
       'id_20', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo'],
      dtype='object') 

starting imputation....

index after imputation:  Int64Index([2987004, 2987008, 2987010, 2987011, 2987016, 2987017, 2987022,
            2987038, 2987040, 2987048,
            ...
            4170224, 4170225, 4170226, 4170228, 4170229, 4170230, 4170233,
            4170234, 41

### Des valeurs manquantes mais pas trop -> on remplace par la médiane


### *Transactions*

In [24]:
print("index before imputation: ", transaction_data.index, "\n")
print("columns before imputation: ", transaction_data.columns, "\n")

print("values before imputing: ", transaction_data[medium_missing_num_trans_cols], "\n")

print("starting imputation.....\n\n")
my_imputer = SimpleImputer(strategy = 'median') #on choisi d'appliquer la médiane
my_imputer.fit(transaction_data[medium_missing_num_trans_cols])

#transformations
transaction_data[medium_missing_num_trans_cols] = my_imputer.transform(transaction_data[medium_missing_num_trans_cols])

print("index after imputation: ", transaction_data.index, "\n")
print("columns after imputation: ", transaction_data.columns, "\n")
#print("values after imputing: ", transaction_data[medium_missing_num_trans_cols], "\n")

index before imputation:  Int64Index([2987000, 2987001, 2987002, 2987003, 2987004, 2987005, 2987006,
            2987007, 2987008, 2987009,
            ...
            4170230, 4170231, 4170232, 4170233, 4170234, 4170235, 4170236,
            4170237, 4170238, 4170239],
           dtype='int64', name='TransactionID', length=1097231) 

columns before imputation:  Index(['TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2',
       'card3', 'card4', 'card5', 'card6', 'addr1',
       ...
       'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320',
       'V321'],
      dtype='object', length=225) 

values before imputing:                 dist1     D2    D3    D4    D5  ...  V48  V49  V50  V51  V52
TransactionID                                  ...                         
2987000         19.0    NaN  13.0   NaN   NaN  ...  NaN  NaN  NaN  NaN  NaN
2987001          NaN    NaN   NaN   0.0   NaN  ...  0.0  0.0  0.0  0.0  0.0
2987002        287.0    NaN   NaN   0.0

### *Identity*

In [25]:
print("index before imputation: ", identity_data.index, "\n")
print("columns before imputation: ", identity_data.columns, "\n")


my_imputer = SimpleImputer(strategy = 'median') 
my_imputer.fit(identity_data[medium_missing_num_id_cols])

print("values before imputing: ", identity_data[medium_missing_num_id_cols], "\n")

#transformations
identity_data[medium_missing_num_id_cols] = my_imputer.transform(identity_data[medium_missing_num_id_cols])

print("index after imputation: ", identity_data.index, "\n")
print("columns after imputation: ", identity_data.columns, "\n")

index before imputation:  Int64Index([2987004, 2987008, 2987010, 2987011, 2987016, 2987017, 2987022,
            2987038, 2987040, 2987048,
            ...
            4170224, 4170225, 4170226, 4170228, 4170229, 4170230, 4170233,
            4170234, 4170236, 4170239],
           dtype='int64', name='TransactionID', length=286140) 

columns before imputation:  Index(['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_09', 'id_10',
       'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_19',
       'id_20', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo'],
      dtype='object') 

values before imputing:                 id_03  id_04  id_09  id_10  id_14  id_32
TransactionID                                          
2987004          NaN    NaN    NaN    NaN -480.0   32.0
2987008          NaN    NaN    NaN    NaN -300.0   32.0
2987010          0.0    0.0  

Il n'y a plus de valeurs numériques manquantes :


In [26]:
print(transaction_data[num_trans_cols].isnull().sum().sum())

0


In [27]:
print(identity_data[num_id_cols].isnull().sum().sum())

0


## On va réduire la taille des dataframes pour éviter d'utiliser trop de RAM

Par défaut toutes les données numériques sont sotckées sur du int_64 ce qui prend beaucoup de place. On va donc les séparées en plusieurs catégories en fonctions de la taille dont elles ont besoin puis réduire la taille qui leur est alloué

**On tri les données en fonction de l'espace dont elles ont besoin**




In [28]:
#fonction générique
# prend un paramèmtre un dataframe et la liste des colonnes dont il faut modifier la taile
# retourne 5 listes contenant le nom des colonnes et indiquant quelle taille leur est necessaire
def detect_num_cols_to_shrink(list_of_num_cols, dataframe):
 
    convert_to_int8 = []
    convert_to_int16 = []
    convert_to_int32 = []
  
    convert_to_float16 = []
    convert_to_float32 = []
    
    for col in list_of_num_cols:
        
        if dataframe[col].dtype in ['int', 'int8', 'int32', 'int64']:
            describe_object = dataframe[col].describe()
            minimum = describe_object[3]
            maximum = describe_object[7]
            diff = abs(maximum - minimum)

            if diff < 255:
                convert_to_int8.append(col)
            elif diff < 65535:
                convert_to_int16.append(col)
            elif diff < 4294967295:
                convert_to_int32.append(col)   
                
        elif dataframe[col].dtype in ['float', 'float16', 'float32', 'float64']:
            describe_object = dataframe[col].describe()
            minimum = describe_object[3]
            maximum = describe_object[7]
            diff = abs(maximum - minimum)

            if diff < 65535:
                convert_to_float16.append(col)
            elif diff < 4294967295:
                convert_to_float32.append(col) 
        
    list_of_lists = []
    list_of_lists.append(convert_to_int8)
    list_of_lists.append(convert_to_int16)
    list_of_lists.append(convert_to_int32)
    list_of_lists.append(convert_to_float16)
    list_of_lists.append(convert_to_float32)
    
    return list_of_lists

### *Transaction*

On tri les données

In [29]:
num_cols_to_shrink_trans = detect_num_cols_to_shrink(num_trans_cols, transaction_data)

convert_to_int8 = num_cols_to_shrink_trans[0]
convert_to_int16 = num_cols_to_shrink_trans[1]
convert_to_int32 = num_cols_to_shrink_trans[2]

convert_to_float16 = num_cols_to_shrink_trans[3]
convert_to_float32 = num_cols_to_shrink_trans[4]

On change la taille qui leur est alloué

In [30]:
print("starting with converting process....")

for col in convert_to_int16:
    transaction_data[col] = transaction_data[col].astype('int16') 
    
for col in convert_to_int32:
    transaction_data[col] = transaction_data[col].astype('int32') 

for col in convert_to_float16:
    transaction_data[col] = transaction_data[col].astype('float16')
    
for col in convert_to_float32:
    transaction_data[col] = transaction_data[col].astype('float32')
    
print("successfully converted!")

starting with converting process....
successfully converted!


### *Identity*

In [31]:
num_cols_to_shrink_id = detect_num_cols_to_shrink(num_id_cols, identity_data)

convert_to_int8 = num_cols_to_shrink_id[0]
convert_to_int16 = num_cols_to_shrink_id[1]
convert_to_int32 = num_cols_to_shrink_id[2]

convert_to_float16 = num_cols_to_shrink_id[3]
convert_to_float32 = num_cols_to_shrink_id[4]

In [32]:
for col in convert_to_float16:
    identity_data[col] = identity_data[col].astype('float16')
    
for col in convert_to_float32:
    identity_data[col] = identity_data[col].astype('float32')
    
    
print("successfully converted!")

successfully converted!


## On transforme les données en fonctions de leur categories : Categorique


### Trop de valeur manquantes --> on supprime les colonnes

### *Transactions*

In [33]:
print("shape before dropping many_missing_cat_trans_cols: ", transaction_data.shape, "\n")        
transaction_data = transaction_data.drop(columns = many_missing_cat_trans_cols)
print("shape after dropping many_missing_cat_trans_cols: ", transaction_data.shape, "\n\n")    

print("shape before dropping many_missing_cat_id_cols: ", identity_data.shape, "\n")        
identity_data = identity_data.drop(columns = many_missing_cat_id_cols)
print("shape after dropping many_missing_cat_id_cols: ", identity_data.shape, "\n") 

shape before dropping many_missing_cat_trans_cols:  (1097231, 225) 

shape after dropping many_missing_cat_trans_cols:  (1097231, 223) 


shape before dropping many_missing_cat_id_cols:  (286140, 32) 

shape after dropping many_missing_cat_id_cols:  (286140, 30) 



on met à jour les listes de nom de colonnes categorielles après suppression

In [34]:
c = (transaction_data.dtypes == 'object')
cat_trans_cols = list(c[c].index) 

c = (identity_data.dtypes == 'object')
cat_id_cols = list(c[c].index)

### Beaucoup mais pas trop de valeurs manquantes --> on remplace les nans par la valeur la plus présente

### *Transaction*

In [35]:
for col in cat_trans_cols:
    print(col, transaction_data[col].nunique(), "\n")

ProductCD 5 

card4 4 

card6 4 

P_emaildomain 60 

M1 2 

M2 2 

M3 2 

M4 3 

M6 2 

M7 2 

M8 2 

M9 2 



On remarque que certaines variables prennent leur valeurs dans un petit ensemble tandis que d'autes prennent leur valeurs dans de plus grand ensembles.

On crée donc 2 listes : 1 avec les colonnes ayant peu de valeur, 1 avec l'autre

In [36]:
low_card_trans_cols = ["ProductCD", "card4", "card6", "M1", "M2", "M3", "M4", "M6", "M7", "M8", "M9"]
high_card_trans_cols = ["P_emaildomain"]

On remplace les valeurs manquantes par la valeur la plus fréquente dans chaque colonnes

In [37]:
for i in cat_trans_cols:
    most_frequent_value = transaction_data[i].mode()[0]
    print("For column: ", i, "the most frequent value is: ", most_frequent_value, "\n")
    transaction_data[i].fillna(most_frequent_value, inplace = True)

For column:  ProductCD the most frequent value is:  W 

For column:  card4 the most frequent value is:  visa 

For column:  card6 the most frequent value is:  debit 

For column:  P_emaildomain the most frequent value is:  gmail.com 

For column:  M1 the most frequent value is:  T 

For column:  M2 the most frequent value is:  T 

For column:  M3 the most frequent value is:  T 

For column:  M4 the most frequent value is:  M0 

For column:  M6 the most frequent value is:  F 

For column:  M7 the most frequent value is:  F 

For column:  M8 the most frequent value is:  F 

For column:  M9 the most frequent value is:  T 



On encode : c'est à dire que l'on donne une valeur numérique à chaque valeur categorielle, pour les variables se trouvant dans high_card_trans_cols

In [38]:
from sklearn.preprocessing import LabelEncoder
    
label_encoder = LabelEncoder()
print("transaction_data.shape before label-encoding: ", transaction_data.shape, "\n")

transaction_data[high_card_trans_cols] = label_encoder.fit_transform(transaction_data[high_card_trans_cols].values.ravel())

print("transaction_data.shape after label-encoding: ", transaction_data.shape, "\n")
print("transaction_data[high_card_trans_cols] after label_encoding: ",transaction_data[high_card_trans_cols], "\n")

transaction_data.shape before label-encoding:  (1097231, 223) 

transaction_data.shape after label-encoding:  (1097231, 223) 

transaction_data[high_card_trans_cols] after label_encoding:                 P_emaildomain
TransactionID               
2987000                   16
2987001                   16
2987002                   35
2987003                   54
2987004                   16
...                      ...
4170235                   16
4170236                   19
4170237                   19
4170238                   19
4170239                   19

[1097231 rows x 1 columns] 



Maintenant on s'intéresse à celles qui ont une plus petite cardinalité

On utilise pd.get_dummies, qui crée des colonnes (de 0 et 1) pour chaque valeur possible que peut prendre une colonne

In [39]:
print("shape before encoding: ", transaction_data.shape, "\n")
print("columns to encode: ", low_card_trans_cols, "\n")
print("transaction_data.columns.to_list() before encoding: ", transaction_data.columns.to_list(), "\n")


low_card_trans_encoded = pd.get_dummies(transaction_data[low_card_trans_cols], dummy_na = False)
transaction_data.drop(columns = low_card_trans_cols, inplace = True)

print("shape after encoding: ", transaction_data.shape, "\n\n")
print("shape of new dataframe: ", low_card_trans_encoded.shape, "\n\n")
print("newly generated columns: ", low_card_trans_encoded.columns, "\n")
print("low_card_trans_encoded.info(): ", low_card_trans_encoded.info(),"\n")
print("transaction_data.columns.to_list() after encoding: ", transaction_data.columns.to_list(), "\n")

shape before encoding:  (1097231, 223) 

columns to encode:  ['ProductCD', 'card4', 'card6', 'M1', 'M2', 'M3', 'M4', 'M6', 'M7', 'M8', 'M9'] 

transaction_data.columns.to_list() before encoding:  ['TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'P_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D10', 'D11', 'D15', 'M1', 'M2', 'M3', 'M4', 'M6', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V

On supprime 1 colonne de chaque nouveau set de colonnes créé pour éviter d'avoir plus de corrélation que nécessaire

In [40]:
print("newly generated columns: ", low_card_trans_encoded.columns, "\n")

low_card_trans_encoded.drop(columns = ['ProductCD_W', 'card4_visa', 'card6_credit', 'M1_F', 'M2_F', 'M3_F', 'M4_M0', 'M6_F', 'M7_F', 'M8_F', 'M9_F'], inplace=True)

print("after deleting one column of each category: ", low_card_trans_encoded.columns, "\n")

newly generated columns:  Index(['ProductCD_C', 'ProductCD_H', 'ProductCD_R', 'ProductCD_S',
       'ProductCD_W', 'card4_american express', 'card4_discover',
       'card4_mastercard', 'card4_visa', 'card6_charge card', 'card6_credit',
       'card6_debit', 'card6_debit or credit', 'M1_F', 'M1_T', 'M2_F', 'M2_T',
       'M3_F', 'M3_T', 'M4_M0', 'M4_M1', 'M4_M2', 'M6_F', 'M6_T', 'M7_F',
       'M7_T', 'M8_F', 'M8_T', 'M9_F', 'M9_T'],
      dtype='object') 

after deleting one column of each category:  Index(['ProductCD_C', 'ProductCD_H', 'ProductCD_R', 'ProductCD_S',
       'card4_american express', 'card4_discover', 'card4_mastercard',
       'card6_charge card', 'card6_debit', 'card6_debit or credit', 'M1_T',
       'M2_T', 'M3_T', 'M4_M1', 'M4_M2', 'M6_T', 'M7_T', 'M8_T', 'M9_T'],
      dtype='object') 



### *Identity*

In [41]:
low_card_id_cols =  ["id_12", "id_15", "id_16", "id_28", "id_29", "id_34", "id_35", "id_36", "id_37", "id_38", "DeviceType"]
high_card_id_cols = ["id_30", "id_31", "id_33", "DeviceInfo"]

In [42]:
for i in cat_id_cols:
    most_frequent_value = identity_data[i].mode()[0]
    print("For column: ", i, "the most frequent value is: ", most_frequent_value, "\n")
    identity_data[i].fillna(most_frequent_value, inplace = True)

For column:  id_12 the most frequent value is:  NotFound 

For column:  id_15 the most frequent value is:  Found 

For column:  id_16 the most frequent value is:  Found 

For column:  id_28 the most frequent value is:  Found 

For column:  id_29 the most frequent value is:  Found 

For column:  id_30 the most frequent value is:  Windows 10 

For column:  id_31 the most frequent value is:  mobile safari 11.0 

For column:  id_33 the most frequent value is:  1920x1080 

For column:  id_34 the most frequent value is:  match_status:2 

For column:  id_35 the most frequent value is:  T 

For column:  id_36 the most frequent value is:  F 

For column:  id_37 the most frequent value is:  T 

For column:  id_38 the most frequent value is:  F 

For column:  DeviceType the most frequent value is:  desktop 

For column:  DeviceInfo the most frequent value is:  Windows 



In [43]:
label_encoder = LabelEncoder()

print("identity_data.shape before label-encoding: ", identity_data.shape, "\n")

for col in high_card_id_cols:
    identity_data[col] = label_encoder.fit_transform(identity_data[col])

print("identity_data.shape after label-encoding: ", identity_data.shape, "\n")
print("identity_data[high_card_id_cols] after label_encoding: ",identity_data[high_card_id_cols], "\n")

identity_data.shape before label-encoding:  (286140, 30) 

identity_data.shape after label-encoding:  (286140, 30) 

identity_data[high_card_id_cols] after label_encoding:                 id_30  id_31  id_33  DeviceInfo
TransactionID                                 
2987004            7    161    268        1565
2987008           70    130     80        2693
2987010           49     46    216        2526
2987011           49     46    216        2526
2987016           26     46     68        1170
...              ...    ...    ...         ...
4170230           49     74    216        2165
4170233           49     74    216        2106
4170234           63    129     80        2693
4170236           49     22    216         141
4170239           49    166    216        1529

[286140 rows x 4 columns] 



Maintenant on s'intéresse à celles qui ont une plus petite cardinalité

On utilise pd.get_dummies, qui crée des colonnes (de 0 et 1) pour chaque valeur possible que peut prendre une colonne

In [44]:
print("shape before encoding: ", identity_data.shape, "\n")
print("columns to encode: ", low_card_id_cols, "\n")


low_card_id_encoded = pd.get_dummies(identity_data[low_card_id_cols], dummy_na = False)
identity_data.drop(columns = low_card_id_cols, inplace = True)


print("shape after encoding: ", identity_data.shape, "\n\n")
print("shape of new dataframe: ", low_card_id_encoded.shape, "\n\n")
print("newly generated columns: ", low_card_id_encoded.columns, "\n")
print("low_card_id_encoded.info(): ", low_card_id_encoded.info())


shape before encoding:  (286140, 30) 

columns to encode:  ['id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType'] 

shape after encoding:  (286140, 19) 


shape of new dataframe:  (286140, 25) 


newly generated columns:  Index(['id_12_Found', 'id_12_NotFound', 'id_15_Found', 'id_15_New',
       'id_15_Unknown', 'id_16_Found', 'id_16_NotFound', 'id_28_Found',
       'id_28_New', 'id_29_Found', 'id_29_NotFound', 'id_34_match_status:-1',
       'id_34_match_status:0', 'id_34_match_status:1', 'id_34_match_status:2',
       'id_35_F', 'id_35_T', 'id_36_F', 'id_36_T', 'id_37_F', 'id_37_T',
       'id_38_F', 'id_38_T', 'DeviceType_desktop', 'DeviceType_mobile'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286140 entries, 2987004 to 4170239
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   id_12_Found            286140 non-nu

On supprime 1 colonne de chaque nouveau set de colonnes créé pour éviter d'avoir plus de corrélation que nécessaire

In [45]:
print("newly generated columns: ", low_card_id_encoded.columns, "\n")

newly generated columns:  Index(['id_12_Found', 'id_12_NotFound', 'id_15_Found', 'id_15_New',
       'id_15_Unknown', 'id_16_Found', 'id_16_NotFound', 'id_28_Found',
       'id_28_New', 'id_29_Found', 'id_29_NotFound', 'id_34_match_status:-1',
       'id_34_match_status:0', 'id_34_match_status:1', 'id_34_match_status:2',
       'id_35_F', 'id_35_T', 'id_36_F', 'id_36_T', 'id_37_F', 'id_37_T',
       'id_38_F', 'id_38_T', 'DeviceType_desktop', 'DeviceType_mobile'],
      dtype='object') 



In [46]:
low_card_id_encoded.drop(columns = ['id_12_Found', 'id_15_Found', 'id_16_Found', 'id_28_Found', 'id_29_Found', 'id_34_match_status:-1', 'id_35_F', 'id_36_F', 'id_37_F', 'id_38_F', 'DeviceType_desktop'], inplace=True)

print("after deleting one column of each category: ", low_card_id_encoded.columns, "\n")

after deleting one column of each category:  Index(['id_12_NotFound', 'id_15_New', 'id_15_Unknown', 'id_16_NotFound',
       'id_28_New', 'id_29_NotFound', 'id_34_match_status:0',
       'id_34_match_status:1', 'id_34_match_status:2', 'id_35_T', 'id_36_T',
       'id_37_T', 'id_38_T', 'DeviceType_mobile'],
      dtype='object') 



### Bilan des transformations

Plus aucune valeurs manquantes dans nos 4 dataframe

In [47]:
print(transaction_data.isnull().sum().sum(), "\n")
print(low_card_trans_encoded.isnull().sum().sum())

0 

0


In [48]:
print(identity_data.isnull().sum().sum(), "\n")
print(low_card_id_encoded.isnull().sum().sum())

0 

0


### Concaténation des tableaux

Maintenant on les concatatène les taleaux afin d'avoir un tableau qui ne contient plus que les données transformées et propre

### *Transactions*

In [49]:
print("transaction_data.shape before concatting: ", transaction_data.shape, "\n")
print("low_card_trans_encoded.shape before concatting: ", low_card_trans_encoded.shape, "\n")

transaction_concatted = pd.concat([transaction_data, low_card_trans_encoded], axis = 1)

print("transaction_concatted.shape after concatting: ", transaction_concatted.shape, "\n")
print("transaction_concatted.columns after concatting: ", transaction_concatted.columns, "\n")

#del low_card_trans_encoded
#del transaction_data

print(transaction_concatted.info())

transaction_data.shape before concatting:  (1097231, 212) 

low_card_trans_encoded.shape before concatting:  (1097231, 19) 

transaction_concatted.shape after concatting:  (1097231, 231) 

transaction_concatted.columns after concatting:  Index(['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5',
       'addr1', 'addr2', 'dist1', 'P_emaildomain',
       ...
       'card6_debit or credit', 'M1_T', 'M2_T', 'M3_T', 'M4_M1', 'M4_M2',
       'M6_T', 'M7_T', 'M8_T', 'M9_T'],
      dtype='object', length=231) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097231 entries, 2987000 to 4170239
Columns: 231 entries, TransactionDT to M9_T
dtypes: float16(188), float32(23), int64(1), uint8(19)
memory usage: 526.3 MB
None


In [50]:
#print(transaction_concatted[['M8_F', 'M8_T']])

### *Identity*

In [51]:
print("identity_data.shape before concatting: ", identity_data.shape, "\n")
print("low_card_id_encoded.shape before concatting: ", low_card_id_encoded.shape, "\n")

identity_concatted = pd.concat([identity_data, low_card_id_encoded], axis = 1)

print("identity_concatted.shape after concatting: ", identity_concatted.shape, "\n")
print("identity_concatted.columns after concatting: ", identity_concatted.columns, "\n")

#del low_card_id_encoded
#del identity_data

print(identity_concatted.info())

identity_data.shape before concatting:  (286140, 19) 

low_card_id_encoded.shape before concatting:  (286140, 14) 

identity_concatted.shape after concatting:  (286140, 33) 

identity_concatted.columns after concatting:  Index(['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_09', 'id_10',
       'id_11', 'id_13', 'id_14', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31',
       'id_32', 'id_33', 'DeviceInfo', 'id_12_NotFound', 'id_15_New',
       'id_15_Unknown', 'id_16_NotFound', 'id_28_New', 'id_29_NotFound',
       'id_34_match_status:0', 'id_34_match_status:1', 'id_34_match_status:2',
       'id_35_T', 'id_36_T', 'id_37_T', 'id_38_T', 'DeviceType_mobile'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286140 entries, 2987004 to 4170239
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id_01                 286140 non-null  float16
 1   id_02                 2

### Maintenant on recrée les dataframes de test et de train

### *Transactions*

In [52]:
print("transaction_concatted.shape before splitting up: ", transaction_concatted.shape, "\n")

# shape of train_transaction was (590540, 393), 
# shape of test_transaction  was (506691, 392)
train_transaction = transaction_concatted.iloc[0:590540]
test_transaction = transaction_concatted.iloc[590540:]

print("train_transaction.shape after splitting up: ", train_transaction.shape, "\n")
print("test_transaction.shape after splitting up: ", test_transaction.shape, "\n")

transaction_concatted.shape before splitting up:  (1097231, 231) 

train_transaction.shape after splitting up:  (590540, 231) 

test_transaction.shape after splitting up:  (506691, 231) 



### *Identity*

In [53]:
print("identity_concatted.shape before splitting up: ", identity_concatted.shape, "\n")

# shape of train_identity was  (144233, 40)
# shape of test_identity  was  (141907, 40)
train_identity = identity_concatted.iloc[0:144233]
test_identity = identity_concatted.iloc[144233:]

print("train_identity.shape after splitting up: ", train_identity.shape, "\n")
print("test_identity.shape after splitting up: ", test_identity.shape, "\n")

identity_concatted.shape before splitting up:  (286140, 33) 

train_identity.shape after splitting up:  (144233, 33) 

test_identity.shape after splitting up:  (141907, 33) 



## On concatène les train ensemble et les test ensembles

### *Train*

In [54]:
print("train_transaction.shape before concatting: ", train_transaction.shape, "\n")
print("train_identity.shape before concatting: ", train_identity.shape, "\n")


train_data  = pd.concat([train_transaction, train_identity], axis = 1)


print("train_data.shape: ", train_data.shape)

train_transaction.shape before concatting:  (590540, 231) 

train_identity.shape before concatting:  (144233, 33) 

train_data.shape:  (590540, 264)


### *Test*

In [55]:
print("test_transaction.shape before concatting: ", test_transaction.shape, "\n")
print("test_identity.shape before concatting: ", test_identity.shape, "\n")

test_data  = pd.concat([test_transaction, test_identity], axis = 1)


print("test_data.shape: ", test_data.shape)

test_transaction.shape before concatting:  (506691, 231) 

test_identity.shape before concatting:  (141907, 33) 

test_data.shape:  (506691, 264)


## On enlève les nouveaux NaN créé en les remplacant tous par la median

In [56]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 2987000 to 3577539
Columns: 264 entries, TransactionDT to DeviceType_mobile
dtypes: float16(202), float32(24), float64(18), int64(1), uint8(19)
memory usage: 382.4 MB


In [57]:
n = (train_data.dtypes != 'object')
num_train_cols = list(n[n].index) 

n = (test_data.dtypes != 'object')
num_test_cols = list(n[n].index) 

In [58]:
my_imputer = SimpleImputer(strategy = 'median') 
my_imputer.fit(train_data[num_train_cols])

train_data[num_train_cols] = my_imputer.transform(train_data[num_train_cols])

my_imputer = SimpleImputer(strategy = 'median') 
my_imputer.fit(test_data[num_test_cols])

test_data[num_test_cols] = my_imputer.transform(test_data[num_test_cols])

In [59]:
print(train_data.shape)
train_data.info()

(590540, 264)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 2987000 to 3577539
Columns: 264 entries, TransactionDT to DeviceType_mobile
dtypes: float64(264)
memory usage: 1.2 GB


In [60]:
print(test_data.shape)
test_data.info()

(506691, 264)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 3663549 to 4170239
Columns: 264 entries, TransactionDT to DeviceType_mobile
dtypes: float64(264)
memory usage: 1.0 GB


In [61]:
print(train_data[num_train_cols].isnull().sum().sum())

0


In [62]:
print(test_data[num_test_cols].isnull().sum().sum())

0


### On cherche à réduire le nombres de variables : PCA

On enlève l'index pour ne pas le modifier

In [63]:
columns = test_data.columns
print(columns)
df = test_data.reset_index(inplace=False)
print(df.head(5))
transactionID = df['TransactionID']
print(transactionID)
print(test_data.head(5))

Index(['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5',
       'addr1', 'addr2', 'dist1', 'P_emaildomain',
       ...
       'id_28_New', 'id_29_NotFound', 'id_34_match_status:0',
       'id_34_match_status:1', 'id_34_match_status:2', 'id_35_T', 'id_36_T',
       'id_37_T', 'id_38_T', 'DeviceType_mobile'],
      dtype='object', length=264)
   TransactionID  TransactionDT  ...  id_38_T  DeviceType_mobile
0        3663549     18403224.0  ...      0.0                0.0
1        3663550     18403264.0  ...      0.0                0.0
2        3663551     18403310.0  ...      0.0                0.0
3        3663552     18403310.0  ...      0.0                0.0
4        3663553     18403316.0  ...      0.0                0.0

[5 rows x 265 columns]
0         3663549
1         3663550
2         3663551
3         3663552
4         3663553
           ...   
506686    4170235
506687    4170236
506688    4170237
506689    4170238
506690    4170239
Name: TransactionID, Len

### On scale les data pour éviter que certaines est une plus gande importance que d'autres


In [64]:
from sklearn.preprocessing import StandardScaler, RobustScaler

on scale les données

In [65]:
train_data = pd.DataFrame(StandardScaler().fit_transform(train_data))
test_data = pd.DataFrame(StandardScaler().fit_transform(test_data))

On reforme les DataFrame car la transformations change leur type; et on remet l'index

In [66]:
print(train_data.head(5))
print(test_data.head(5))
print(train_data.shape)
print(test_data.shape)
train_data.columns = columns
test_data.columns = columns

train_data  = pd.concat([train_data, transactionID], axis = 1)
test_data  = pd.concat([test_data, transactionID], axis = 1)

train_data.set_index(['TransactionID'], inplace=True)
test_data.set_index(['TransactionID'], inplace=True)

print(train_data.head(5))
print(test_data.head(5))
print(train_data.shape)
print(test_data.shape)

        0         1         2    ...       261       262       263
0 -1.577987 -0.278174  0.822123  ...  0.233501 -0.357926 -0.322536
1 -1.577986 -0.443337 -1.457409  ...  0.233501 -0.357926 -0.322536
2 -1.577972 -0.317897 -1.068101  ...  0.233501 -0.357926 -0.322536
3 -1.577965 -0.355529  1.679090  ...  0.233501 -0.357926 -0.322536
4 -1.577964 -0.355529 -1.102380  ...  0.233501  2.793877  3.100425

[5 rows x 264 columns]
        0         1         2    ...       261       262       263
0 -1.792644 -0.418154  0.092274  ...  0.260849 -0.300321 -0.375203
1 -1.792635 -0.348795 -1.163865  ...  0.260849 -0.300321 -0.375203
2 -1.792626  0.147590 -1.122103  ...  0.260849 -0.300321 -0.375203
3 -1.792626  0.611425  0.211828  ...  0.260849 -0.300321 -0.375203
4 -1.792624 -0.271743  1.649755  ...  0.260849 -0.300321 -0.375203

[5 rows x 264 columns]
(590540, 264)
(506691, 264)
               TransactionDT  TransactionAmt  ...   id_38_T  DeviceType_mobile
TransactionID                            

### PCA

In [67]:
from sklearn.decomposition import PCA
pca = PCA()
x_pca = pd.DataFrame(pca.fit_transform(train_data ))

In [68]:
my_var = pca.explained_variance_ratio_

In [69]:
sum(my_var[0:135])

0.9744200872537422

On atteind 97% des données avec les 135premiere variables (sur 264 variables) , on peut donc supprimer les autres

In [70]:
#99% des données
short_train_165 = train_data.iloc[:,0:165]
short_test_165 = test_data.iloc[:,0:165]

In [71]:
#97% des données
short_train_135 = train_data.iloc[:,0:135]
short_test_135 = test_data.iloc[:,0:135]

In [72]:
#95% des données
short_train_115 = train_data.iloc[:,0:115]
short_test_115 = test_data.iloc[:,0:115]

In [73]:
#90% des données
short_train_90 = train_data.iloc[:,0:90]
short_test_90 = test_data.iloc[:,0:90]

## On sauvegarde les nouvelles données, scaled, sans Nan, et dont on a supprimé certaines colonnes



In [ ]:
short_train_165.to_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/short_train99.csv')
short_test_165.to_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/short_test99.csv')

In [ ]:
short_train_135.to_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/short_train97.csv')
short_test_135.to_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/short_test97.csv')

In [ ]:
short_train_115.to_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/short_train95.csv')
short_test_115.to_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/short_test95.csv')

In [ ]:
short_train_90.to_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/short_train90.csv')
short_test_90.to_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/short_test90.csv')

In [ ]:
y_train.to_csv('/content/drive/MyDrive/Colab Notebooks/MOST_AA/données/Y_train.csv')

Reste à train un modèle